<a href="https://colab.research.google.com/github/dropcreations/FFmpeg-in-Google-Colab/blob/main/FFmpeg-in-Google-Drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# __Mount Google Drive__

In [ ]:
#@markdown <br><center><img src='https://raw.githubusercontent.com/dropcreations/FFmpeg-for-Google-Colab/1bd5d8aeb88c8f402a9aea4846fab6304e7d8e43/Google-Drive-Logo.svg' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3><b>Mount Google Drive</b></h3></center><br>

from google.colab import drive

mode = "Mount" #@param ["Mount", "Unmount"]
drive.mount._DEBUG = False

if mode == "Mount":
    drive.mount('/content/drive', force_remount=True)
elif mode == "Unmount":
    try: drive.flush_and_unmount()
    except ValueError: pass
    !rm -rf /root/.config/Google/DriveFS

# __Install FFmpeg__

In [ ]:
#@markdown <br><center><img src='https://raw.githubusercontent.com/dropcreations/FFmpeg-for-Google-Colab/32abf44ff4c8d145a94a24611f01141926a8daaa/FFmpeg-Logo.svg' height="40" alt="FFmpeg-logo"/></center>
#@markdown <center><h3><b>Install FFmpeg</b></h3></center><br>
from IPython.display import clear_output
!sudo curl -L https://github.com/BtbN/FFmpeg-Builds/releases/download/latest/ffmpeg-master-latest-linux64-gpl.tar.xz -o /usr/local/bin/ffmpeg.tar.xz
clear_output()
%cd /usr/local/bin/
clear_output()
!7z e /usr/local/bin/ffmpeg.tar.xz
clear_output()
!7z e /usr/local/bin/ffmpeg.tar
clear_output()
!sudo chmod a+rx /usr/local/bin/ffmpeg
clear_output()
%cd /content/
!sudo curl -L https://mkvtoolnix.download/appimage/MKVToolNix_GUI-70.0.0-x86_64.AppImage -o /usr/local/bin/MKVToolNix_GUI-70.0.0-x86_64.AppImage
!sudo chmod u+rx /usr/local/bin/MKVToolNix_GUI-70.0.0-x86_64.AppImage
!sudo ln -s /usr/local/bin/MKVToolNix_GUI-70.0.0-x86_64.AppImage /usr/local/bin/mkvmerge
!sudo chmod a+rx /usr/local/bin/mkvmerge
clear_output()
!ffmpeg -version

### __Remux Video Files__

* **REMUX** video files without **RE-ENCODING**.
* Make sure that `outputFormat` that you have selected is supported for all tracks in the `inputFile`.
* `WebM` only support `VP9`, `VP8`, `AV1` video and `Vorbis`, `Opus` audio and `WebVTT` subtitles.

In [ ]:
import os

inputFile = "" #@param {type:"string"}
outputFormat = "mp4" #@param ["mp4", "mkv", "mov", "webm"]
saveToSourceLocation = True #@param {type:"boolean"}

commandLine = '-hide_banner -i "' + inputFile + '" -map 0:v? -c:v copy -map 0:a? -c:a copy '

if outputFormat == "mp4" or outputFormat == "mov":
    commandLine = commandLine + '-map 0:s? -c:s mov_text -movflags +faststart '
elif outputFormat == "webm":
    commandLine = commandLine + '-map 0:s? -c:s webvtt '

sourceName = os.path.splitext(os.path.basename(os.path.abspath(inputFile)))[0]
sourceFolder = os.path.dirname(os.path.abspath(inputFile))

if saveToSourceLocation:
    outputFile = os.path.join(sourceFolder, (sourceName + '.' + outputFormat))
    if os.path.isfile(outputFile):
        outputFile = os.path.join(sourceFolder, (sourceName + '_[REMUX].' + outputFormat))
    commandLine = commandLine + '"' + outputFile + '"'
else:
    outputFolder = input("outputFolder: ")
    outputFile = os.path.join(outputFolder, (sourceName + '.' + outputFormat))
    if os.path.isfile(outputFile):
        outputFile = os.path.join(outputFolder, (sourceName + '_[REMUX].' + outputFormat))
    commandLine = commandLine + '"' + outputFile + '"'

!ffmpeg {commandLine}

### __Trim Video Files__

* **TRIM** video files without **RE-ENCODING**.
* `inputFile`: video file's path and set `startTime` and `endTime` to trim.
* Trimed video will be saved as same as the **source's format**.

In [ ]:
import os

inputFile = "" #@param {type:"string"}
startTime = "00:00:00.000" #@param {type:"string"}
endTime = "00:00:00.000" #@param {type:"string"}
saveToSourceLocation = True #@param {type:"boolean"}

commandLine = '-hide_banner -i "' + inputFile + '" -map 0 -c copy -ss ' + startTime + ' -to ' + endTime + ' '

sourceName = os.path.splitext(os.path.basename(os.path.abspath(inputFile)))[0]
sourceExtention = os.path.splitext(os.path.basename(os.path.abspath(inputFile)))[1]
sourceFolder = os.path.dirname(os.path.abspath(inputFile))

if saveToSourceLocation:
    outputFile = os.path.join(sourceFolder, (sourceName + f'_[{startTime}-{endTime}]' + sourceExtention))
    commandLine = commandLine + '"' + outputFile + '"'
else:
    outputFolder = input("outputFolder: ")
    outputFile = os.path.join(outputFolder, (sourceName + f'_[{startTime}-{endTime}]' + sourceExtention))
    commandLine = commandLine + '"' + outputFile + '"'

!ffmpeg {commandLine}

### __Extract Audio__

* **EXTRACT** audio tracks from video files.
* `DTS`, `DTS-HD`, `DTS-MA`, `TrueHD` tracks will be muxed as a `.mka` file.

In [ ]:
import os
import json
import subprocess
import prettytable

inputFile = "" #@param {type:"string"}
saveToSourceLocation = True #@param {type:"boolean"}

jsonFFprobe = subprocess.check_output([
    'ffprobe',
    '-hide_banner',
    '-loglevel',
    'warning',
    '-print_format',
    'json',
    '-show_entries',
    'stream',
    '-i',
    os.path.abspath(inputFile),
    ], stderr=subprocess.DEVNULL)
jsonData = json.loads(jsonFFprobe)

streamCount = len(jsonData['streams'])
commandLine = '-hide_banner -i "' + inputFile + '" '

codecTable = prettytable.PrettyTable()
codecTable.field_names = ['Track ID', 'Codec name', 'Channels', 'Sample rate']
codecTable.align = 'l'

for i in range(streamCount):
    codec_type = jsonData.get('streams')[int(i)].get('codec_type')
    if codec_type == "audio":
        index = jsonData.get('streams')[int(i)].get('index')
        codec_name = jsonData.get('streams')[int(i)].get('codec_name')
        channel_layout = jsonData.get('streams')[int(i)].get('channels')
        sample_rate = jsonData.get('streams')[int(i)].get('sample_rate')
        codecTable.add_row([index, codec_name, channel_layout, sample_rate])

stringTable = '\t' + codecTable.__str__().replace('\n', '\n\t')
print(stringTable)

trackID = input("\ntrackID: ")
codec_name = jsonData.get('streams')[int(trackID)].get('codec_name')
if "opus" in codec_name:
    codec_ext = "ogg"
elif "dts" in codec_name:
    codec_ext = "mka"
elif "aac" in codec_name:
    codec_ext = "aac"
elif "truehd" in codec_name:
    codec_ext = "mka"
elif codec_name == "ac3":
    codec_ext = "ac3"
elif codec_name == "eac3":
    codec_ext = "eac3"
elif "pcm" in codec_name:
    codec_ext = "wav"
elif "flac" in codec_name:
    codec_ext = "flac"

commandLine = commandLine + "-vn -sn -map 0:" + str(trackID) + " -c copy "

sourceName = os.path.splitext(os.path.basename(os.path.abspath(inputFile)))[0]
sourceFolder = os.path.dirname(os.path.abspath(inputFile))

if saveToSourceLocation:
    outputFile = os.path.join(sourceFolder, (sourceName + f'_[{codec_name}].' + codec_ext))
    commandLine = commandLine + '"' + outputFile + '"'
else:
    outputFolder = input("outputFolder: ")
    outputFile = os.path.join(outputFolder, (sourceName + f'_[{codec_name}].' + codec_ext))
    commandLine = commandLine + '"' + outputFile + '"'

!ffmpeg {commandLine}

### __Remove Bitstream Metadata__

* For `H.264`/`AVC` tracks run below cell.

In [ ]:
import os
import json
import subprocess

inputFile = "" #@param {type:"string"}
saveToSourceLocation = True #@param {type:"boolean"}

jsonFFprobe = subprocess.check_output([
    'ffprobe',
    '-hide_banner',
    '-loglevel',
    'warning',
    '-print_format',
    'json',
    '-show_entries',
    'stream',
    '-i',
    os.path.abspath(inputFile),
    ], stderr=subprocess.DEVNULL)
jsonData = json.loads(jsonFFprobe)

streamCount = len(jsonData['streams'])
commandLine = '-hide_banner -i "' + inputFile + '" '

for i in range(streamCount):
    codec_name = jsonData.get('streams')[int(i)].get('codec_name')
    if codec_name == "h264":
        index = jsonData.get('streams')[int(i)].get('index')
        commandLine = commandLine + f'-map 0:{index} '

commandLine = commandLine + '-c:v copy -map 0:a? -c:a copy -map 0:s? -c:s copy -bitexact -map_metadata -1 -vbsf filter_units=remove_types=6 '

sourceName = os.path.splitext(os.path.basename(os.path.abspath(inputFile)))[0]
sourceExtention = os.path.splitext(os.path.basename(os.path.abspath(inputFile)))[1]
sourceFolder = os.path.dirname(os.path.abspath(inputFile))

if saveToSourceLocation:
    outputFile = os.path.join(sourceFolder, (sourceName + f'_[NEW]' + sourceExtention))
    commandLine = commandLine + '"' + outputFile + '"'
else:
    outputFolder = input("outputFolder: ")
    outputFile = os.path.join(outputFolder, (sourceName + f'_[NEW]' + sourceExtention))
    commandLine = commandLine + '"' + outputFile + '"'

!ffmpeg {commandLine}

* For `H.265`/`HEVC` tracks run below cell.

In [ ]:
import os
import json
import subprocess
from IPython.display import clear_output

inputFile = "" #@param {type:"string"}
saveToSourceLocation = True #@param {type:"boolean"}

jsonFFprobe = subprocess.check_output([
    'ffprobe',
    '-hide_banner',
    '-loglevel',
    'warning',
    '-print_format',
    'json',
    '-show_entries',
    'stream',
    '-i',
    os.path.abspath(inputFile),
    ], stderr=subprocess.DEVNULL)
jsonData = json.loads(jsonFFprobe)

streamCount = len(jsonData['streams'])
commandLine = f'-hide_banner -i "{inputFile}" '
tracksALL = []
tracksHEVC = []

for i in range(streamCount):
    indexALL = jsonData.get('streams')[int(i)].get('index')
    tracksALL.append(indexALL)
    codec_name = jsonData.get('streams')[int(i)].get('codec_name')
    if codec_name == "hevc":
        index = jsonData.get('streams')[int(i)].get('index')
        tracksHEVC.append(index)

commandLine = commandLine + f'-map 0:{tracksHEVC[0]} -c:v copy -an -sn -vbsf hevc_mp4toannexb /content/source-video.hevc && ffmpeg -hide_banner -i /content/source-video.hevc -c copy -vbsf filter_units=remove_types=39 /content/output.hevc && mkvmerge --output /content/output.mkv /content/output.hevc && ffmpeg -hide_banner -i /content/output.mkv -i "{inputFile}" -map 0:0 '
for trackID in tracksALL:
    if trackID != tracksHEVC[0]:
        commandLine = commandLine + f'-map 1:{trackID} '
commandLine = commandLine + f'-c copy '

sourceName = os.path.splitext(os.path.basename(os.path.abspath(inputFile)))[0]
sourceExtention = os.path.splitext(os.path.basename(os.path.abspath(inputFile)))[1]
sourceFolder = os.path.dirname(os.path.abspath(inputFile))

if saveToSourceLocation:
    outputFile = os.path.join(sourceFolder, (sourceName + f'_[NEW]' + sourceExtention))
    commandLine = commandLine + f'"{outputFile}"'
else:
    outputFolder = input("outputFolder: ")
    outputFile = os.path.join(outputFolder, (sourceName + f'_[NEW]' + sourceExtention))
    commandLine = commandLine + f'"{outputFile}"'

!ffmpeg {commandLine}
!rm /content/source-video.hevc
!rm /content/output.hevc
!rm /content/output.mkv

### __Encode H.264__

* `CRF` and `2-Pass` encoding available.
* This cell only encodes the **first video track** from the input file.

In [ ]:
inputFile = ""  #@param {type:"string"}
encodeMode = "CRF" #@param ["CRF", "2-Pass"]
saveToSourceLocation = True #@param {type:"boolean"}
#@markdown ### __CRF Settings__
crf = 19 #@param {type:"slider", min:0, max:51, step:1}
#@markdown ### __2-Pass Settings__
Bitrate = "5800" #@param {type:"string"}
#@markdown ### __General Settings__
Preset = "veryslow" #@param ["ultrafast", "superfast", "veryfast", "faster", "fast", "medium", "slow", "slower", "veryslow", "placebo"]
Profile = "high" #@param ["baseline", "main", "high", "high10", "high422", "high444"]
Level = "4" #@param ["1", "1.1", "1.2", "1.3", "2", "2.1", "2.2", "3", "3.1","3.2", "4", "4.1", "4.2", "5", "5.1", "5.2"]
bitDepth = "8-Bit" #@param ["8-Bit", "10-Bit"]
Tune = "None" #@param ["None", "film", "animation", "grain", "stillimage", "psnr", "ssim", "fastdecode", "zerolatency"]
Maxrate = "8700" #@param {type:"string"}
Bufsize = "11600" #@param {type:"string"}

import os

commandLine = f'-preset {Preset} -profile:v {Profile} -level {Level} -maxrate {Maxrate}k -bufsize {Bufsize}k '

if Tune != "None":
    commandLine = commandLine + f'-tune {Tune} '
if bitDepth == "8-Bit":
    commandLine = commandLine + f'-pix_fmt yuv420p '
elif bitDepth == "10-Bit":
    commandLine = commandLine + f'-pix_fmt yuv420p10le '

commandLine = commandLine + f'-partitions all -me_range 24 -nal-hrd 1 -sc_threshold 0 -x264-params "colorprim=bt709:colormatrix=bt709:transfer=bt709:qpmin=6:qpmax=51:bframes=4:lookahead-threads=2:stitchable=1"'

sourceName = os.path.splitext(os.path.basename(os.path.abspath(inputFile)))[0]
sourceExtention = os.path.splitext(os.path.basename(os.path.abspath(inputFile)))[1]
sourceFolder = os.path.dirname(os.path.abspath(inputFile))

if saveToSourceLocation:
    outputFile = os.path.join(sourceFolder, (sourceName + f'_[Encoded]' + sourceExtention))
else:
    outputFolder = input("outputFolder: ")
    outputFile = os.path.join(outputFolder, (sourceName + f'_[Encoded]' + sourceExtention))

firstPass = f'-hide_banner -i "{os.path.abspath(inputFile)}" -map 0:v:0 -c:v libx264 -pass 1 -b:v {Bitrate}k {commandLine} -an -sn -y -f null /dev/null'
secondPass = f'-hide_banner -i "{os.path.abspath(inputFile)}" -map 0:v:0 -c:v libx264 -pass 2 -b:v {Bitrate}k {commandLine} -map 0:a? -c:a copy -map 0:s? -c:s copy "{outputFile}"'
crfPass = f'-hide_banner -i "{os.path.abspath(inputFile)}" -map 0:v:0 -c:v libx264 -crf {crf} {commandLine} -map 0:a? -c:a copy -map 0:s? -c:s copy "{outputFile}"'

if encodeMode == 'CRF':
    !ffmpeg {crfPass}
else:
    !ffmpeg {firstPass}
    !ffmpeg {secondPass}

### __HDR to SDR__

* Convert `HDR` video to `SDR` video.
* `toneMap`: Select tone mapping method.

In [ ]:
import os

inputFile = "" #@param {type:"string"}
Codec = "HEVC" #@param ["H.264", "HEVC"]
valueCRF = 22 #@param {type:"slider", min:0, max:51, step:1}
bitDepth = "10-Bit" #@param ["8-Bit", "10-Bit"]
toneMap = "hable" #@param ["none", "clip", "linear", "gamma", "reinhard", "hable", "mobius"]
saveToSourceLocation = True #@param {type:"boolean"}

if Codec == "H.264":
  Codec = "libx264"
elif Codec == "HEVC":
  Codec = "libx265"

if bitDepth == "8-Bit":
    bitDepth = "yuv420p"
elif bitDepth == "10-Bit":
    bitDepth = "yuv420p10le"

commandLine = f'-hide_banner -i "{inputFile}" -max_muxing_queue_size 1024 -c:v {Codec} -crf:v {valueCRF} -preset:v medium -pix_fmt {bitDepth} '

if Codec == "libx265":
    commandLine = commandLine + f'-x265-params "aq-mode=2:repeat-headers=0:strong-intra-smoothing=1:bframes=4:b-adapt=2:frame-threads=0" '
commandLine = commandLine + f'-vf zscale=t=linear:npl=100,format=gbrpf32le,zscale=p=bt709,tonemap=tonemap={toneMap}:desat=0,zscale=t=bt709:m=bt709:r=tv,format={bitDepth} -map 0:a? -c:a copy -map 0:s? -c:s copy '

sourceName = os.path.splitext(os.path.basename(os.path.abspath(inputFile)))[0]
sourceExtention = os.path.splitext(os.path.basename(os.path.abspath(inputFile)))[1]
sourceFolder = os.path.dirname(os.path.abspath(inputFile))

if saveToSourceLocation:
    outputFile = os.path.join(sourceFolder, (sourceName + f'_[SDR]' + sourceExtention))
    commandLine = commandLine + '"' + outputFile + '"'
else:
    outputFolder = input("outputFolder: ")
    outputFile = os.path.join(outputFolder, (sourceName + f'_[SDR]' + sourceExtention))
    commandLine = commandLine + '"' + outputFile + '"'

!ffmpeg {commandLine}

### __HDR10 Encoding__

* Only `x265`/`HEVC` tracks are supported.
* `extractColors`: View the color information of the footage.
* When encoding, make sure `extractColors` is **deselected**.

In [ ]:
import os
import json
import subprocess

inputFile = "" #@param {type:"string"}
preset = "slow" #@param ["ultrafast", "superfast", "veryfast", "faster", "fast", "medium", "slow", "slower", "veryslow", "placebo"]
profile = "main10" #@param ["main", "main10"]
level = "4" #@param ["1", "1.1", "1.2", "1.3", "2", "2.1", "2.2", "3", "3.1","3.2", "4", "4.1", "4.2", "5", "5.1", "5.2"]
valueCRF = 19 #@param {type:"slider", min:0, max:51, step:1}
extractColors = False #@param {type:"boolean"}
saveToSourceLocation = True #@param {type:"boolean"}

jsonFFprobe = subprocess.check_output([
    'ffprobe', '-hide_banner', '-loglevel', 'warning',
    '-print_format', 'json', '-show_frames', '-read_intervals', '%+#1', '-show_entries',
    'format:stream:frame=color_space,color_primaries,color_transfer,side_data_list,pix_fmt',
    '-i', os.path.abspath(inputFile)], stderr=subprocess.DEVNULL)
jsonData = json.loads(jsonFFprobe)

pix_fmt = jsonData.get('frames')[0].get('pix_fmt')

if jsonData.get('frames')[0].get('color_space'):
    color_space = jsonData.get('frames')[0].get('color_space')
if jsonData.get('frames')[0].get('color_primaries'):
    color_primaries = jsonData.get('frames')[0].get('color_primaries')
if jsonData.get('frames')[0].get('color_transfer'):
    color_transfer = jsonData.get('frames')[0].get('color_transfer')
if jsonData.get('frames')[0].get('side_data_list'):
    side_data_list = jsonData.get('frames')[0].get('side_data_list')[0]

    red_x = round(eval(side_data_list.get('red_x', '0')), 4)
    red_y = round(eval(side_data_list.get('red_y', '0')), 4)
    green_x = round(eval(side_data_list.get('green_x', '0')), 4)
    green_y = round(eval(side_data_list.get('green_y', '0')), 4)
    blue_x = round(eval(side_data_list.get('blue_x', '0')), 4)
    blue_y = round(eval(side_data_list.get('blue_y', '0')), 4)
    white_point_x = round(eval(side_data_list.get('white_point_x', '0')), 4)
    white_point_y = round(eval(side_data_list.get('white_point_y', '0')), 4)
    min_luminance = round(eval(side_data_list.get('min_luminance', '0')), 4)
    max_luminance = round(eval(side_data_list.get('max_luminance', '0')), 4)

    cll_max_content = 0
    cll_max_average = 0

    try:
        side_data_list = jsonData.get('frames')[0].get('side_data_list')[1]
        cll_max_content = round(eval(str(side_data_list.get('max_content', '0'))), 4)
        cll_max_average = round(eval(str(side_data_list.get('max_average', '0'))), 4)
    except:
        pass

    x265_red_x = round(red_x / 0.00002)
    x265_red_y = round(red_y / 0.00002)
    x265_green_x = round(green_x / 0.00002)
    x265_green_y = round(green_y / 0.00002)
    x265_blue_x = round(blue_x / 0.00002)
    x265_blue_y = round(blue_y / 0.00002)
    x265_white_point_x = round(white_point_x / 0.00002)
    x265_white_point_y = round(white_point_y / 0.00002)
    x265_min_luminance = round(min_luminance / 0.0001)
    x265_max_luminance = round(max_luminance / 0.0001)

    x265_master_display_info = 'G(' + str(x265_green_x) + ',' + str(x265_green_y) + ')B(' \
        + str(x265_blue_x) + ',' + str(x265_blue_y) + ')R(' + str(x265_red_x) + ',' \
        + str(x265_red_y) + ')WP(' + str(x265_white_point_x) + ',' \
        + str(x265_white_point_y) + ')L(' + str(x265_max_luminance) + ',' \
        + str(x265_min_luminance) + '):max-cll=' + str(cll_max_content) \
        + ',' + str(cll_max_average)

commandLine = f'-hide_banner -i "{os.path.abspath(inputFile)}" -map 0:v -c:v libx265 -crf {valueCRF} -preset {preset} '

if pix_fmt == "rgb24":
    print("You have selected a Non-HDR footage....")
else:
    if extractColors:
        if jsonData.get('frames')[0].get('color_space'):
            print("color_space: \t\t" + color_space)
        if jsonData.get('frames')[0].get('color_primaries'):
            print("color_primaries: \t" + color_primaries)
        if jsonData.get('frames')[0].get('color_transfer'):
            print("color_transfer: \t" + color_transfer)
        if jsonData.get('frames')[0].get('side_data_list'):
            print("master_display:\t\t" + x265_master_display_info)
    else:
        if profile == "main":
            commandLine = commandLine + f'-profile:v main -pix_fmt yuv420p '
        elif profile == "main10":
            commandLine = commandLine + f'-profile:v main10 -pix_fmt yuv420p10le '
        if jsonData.get('frames')[0].get('side_data_list'):
            commandLine = commandLine + f'-x265-params "level-idc={level}:aq-mode=2:no-info=1:strong-intra-smoothing=1:bframes=4:b-adapt=2:frame-threads=0:hdr10=1:chromaloc=2:repeat-headers=1:aud=1:hrd=1:colorprim={color_primaries}:colormatrix={color_space}:transfer={color_transfer}:range=limited:master-display={x265_master_display_info}" '
        else:
            commandLine = commandLine + f'-x265-params "level-idc={level}:aq-mode=2:no-info=1:strong-intra-smoothing=1:bframes=4:b-adapt=2:frame-threads=0:chromaloc=2:repeat-headers=1:colorprim={color_primaries}:colormatrix={color_space}:transfer={color_transfer}:range=limited" '
        commandLine = commandLine + f'-map 0:a? -c:a copy -map 0:s? -c:s copy '

        sourceName = os.path.splitext(os.path.basename(os.path.abspath(inputFile)))[0]
        sourceExtention = os.path.splitext(os.path.basename(os.path.abspath(inputFile)))[1]
        sourceFolder = os.path.dirname(os.path.abspath(inputFile))

        if saveToSourceLocation:
            outputFile = os.path.join(sourceFolder, (sourceName + f'_[HDR]' + sourceExtention))
            commandLine = commandLine + '"' + outputFile + '"'
        else:
            outputFolder = input("outputFolder: ")
            outputFile = os.path.join(outputFolder, (sourceName + f'_[HDR]' + sourceExtention))
            commandLine = commandLine + '"' + outputFile + '"'

        !ffmpeg {commandLine}